# Oak Wilt Data Cleaning

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Original Data

In [2]:
# Load the original dataset
df = pd.read_csv('../data/data_original.csv')

print(f"Original dataset loaded!")
print(f"Total records: {len(df):,}")
print(f"\nFirst 5 rows:")
df.head()

Original dataset loaded!
Total records: 2,091

First 5 rows:


,Inspection_ID (unique),MORTALITY_CENTER_ID,INSPECTION_DATE,INSPECTION_DECADE,INSPECTION_YEAR,SPECIES,LATITUDE,LONGITUDE,LANDOWNER_TYPE,OAK_WILT_CONFIRMED,SYMPTOM_VEINAL_NECROSIS,SYMPTOM_MORTALITY_PATTERN
1,NaN,5758,8/24/86,1980s,1986,Live Oak,30.318636,-97.766907,Unknown,Yes,<Null>,<Null>
2,NaN,5723,3/11/87,1980s,1987,Live Oak,30.263226,-97.804992,Unknown,Yes,Yes,<Null>
3,NaN,5782,3/17/88,1980s,1988,Live Oak,30.247574,-97.759786,Unknown,Yes,<Null>,<Null>
4,NaN,5807,5/6/88,1980s,1988,Live Oak,30.229521,-97.859917,Unknown,Yes,Yes,<Null>
5,NaN,5689,12/6/88,1980s,1988,Live Oak,30.388368,-97.687027,Unknown,Yes,Yes,<Null>


## 3. Examine Current Values

In [3]:
# Check OAK_WILT_CONFIRMED values before cleaning
print("="*60)
print("OAK_WILT_CONFIRMED - BEFORE CLEANING")
print("="*60)
print(df['OAK_WILT_CONFIRMED'].value_counts())
print(f"\nTotal: {len(df):,}")
print()

OAK_WILT_CONFIRMED - BEFORE CLEANING
OAK_WILT_CONFIRMED
Yes             1735
Undetermined     331
No                12
Maybe             12
Nearby             1
Name: count, dtype: int64

Total: 2,091



In [4]:
# Check SPECIES values before cleaning
print("="*60)
print("SPECIES - BEFORE CLEANING (Top 20)")
print("="*60)
print(df['SPECIES'].value_counts().head(20))
print(f"\nTotal unique species: {df['SPECIES'].nunique()}")
print(f"Total records: {len(df):,}")

SPECIES - BEFORE CLEANING (Top 20)
SPECIES
Live Oak                          1660
Live Oaks                          118
Live Oak, Red Oak                   92
<Null>                              91
Red Oak                             72
Shumard Oak                         13
Live Oak, Shumard Oak               13
Live Oaks, Red Oaks                  8
Live Oak, Blackjack Oak              6
Live Oaks, Red Oak                   6
Live Oak, Pin Oak                    4
Various Oaks                         2
Blackjack Oak                        2
Red Oaks                             1
Live Oaks, Shumard Oak               1
Blackjack Oak, Shumard Oak           1
Live Oak, Red Oak, Shumard Oak       1
Name: count, dtype: int64

Total unique species: 17
Total records: 2,091


## 4. Clean OAK_WILT_CONFIRMED Column

In [5]:
# Step 1: Change 'Nearby' to 'Yes'
nearby_count = (df['OAK_WILT_CONFIRMED'] == 'Nearby').sum()
df['OAK_WILT_CONFIRMED'] = df['OAK_WILT_CONFIRMED'].replace('Nearby', 'Yes')
print(f"Changed {nearby_count:,} 'Nearby' values to 'Yes'")

# Step 2: Remove rows with 'Undetermined' or 'Maybe'
before_removal = len(df)
df = df[~df['OAK_WILT_CONFIRMED'].isin(['Undetermined', 'Maybe'])]
removed = before_removal - len(df)
print(f"Removed {removed:,} rows with 'Undetermined' or 'Maybe'")
print(f"Records remaining: {len(df):,}")

# Verify the results
print("\n" + "="*60)
print("OAK_WILT_CONFIRMED - AFTER CLEANING")
print("="*60)
print(df['OAK_WILT_CONFIRMED'].value_counts())

Changed 1 'Nearby' values to 'Yes'
Removed 343 rows with 'Undetermined' or 'Maybe'
Records remaining: 1,748

OAK_WILT_CONFIRMED - AFTER CLEANING
OAK_WILT_CONFIRMED
Yes    1736
No       12
Name: count, dtype: int64


## 5. Filter Species - Keep Only Live Oak

In [6]:
# Replace <Null> with NaN for proper handling
df = df.replace('<Null>', np.nan)

# Filter to keep only rows where SPECIES contains 'Live Oak'
# This includes: 'Live Oak', 'Live Oaks', 'Live Oak, Red Oak', etc.
before_filter = len(df)

# Keep rows where SPECIES contains 'Live Oak' (case-insensitive)
df = df[df['SPECIES'].str.contains('Live Oak', case=False, na=False)]

removed = before_filter - len(df)
print(f"Removed {removed:,} rows without 'Live Oak' in SPECIES")
print(f"Records remaining: {len(df):,}")

# Verify the results
print("\n" + "="*60)
print("SPECIES - AFTER CLEANING (All unique values)")
print("="*60)
print(df['SPECIES'].value_counts())
print(f"\nTotal unique species combinations: {df['SPECIES'].nunique()}")

Removed 66 rows without 'Live Oak' in SPECIES
Records remaining: 1,682

SPECIES - AFTER CLEANING (All unique values)
SPECIES
Live Oak                          1497
Live Oaks                           97
Live Oak, Red Oak                   60
Live Oaks, Red Oaks                  7
Live Oak, Shumard Oak                7
Live Oak, Blackjack Oak              5
Live Oak, Pin Oak                    4
Live Oaks, Red Oak                   4
Live Oak, Red Oak, Shumard Oak       1
Name: count, dtype: int64

Total unique species combinations: 9


## 6. Summary of Changes

In [7]:
# Load original data for comparison
df_original = pd.read_csv('../data/data_original.csv')

print("="*60)
print("CLEANING SUMMARY")
print("="*60)
print(f"Original records:        {len(df_original):,}")
print(f"Cleaned records:         {len(df):,}")
print(f"Records removed:         {len(df_original) - len(df):,}")
print(f"Percentage retained:     {len(df) / len(df_original) * 100:.1f}%")
print()
print("Changes made:")
print("  ✓ Changed 'Nearby' → 'Yes' in OAK_WILT_CONFIRMED")
print("  ✓ Removed 'Undetermined' and 'Maybe' from OAK_WILT_CONFIRMED")
print("  ✓ Kept only rows with 'Live Oak' in SPECIES")
print()
print("Final OAK_WILT_CONFIRMED values:")
print(df['OAK_WILT_CONFIRMED'].value_counts())
print()
print("Final data quality:")
print(f"  - All rows have Yes/No for oak wilt confirmation")
print(f"  - All rows contain Live Oak trees")

CLEANING SUMMARY
Original records:        2,091
Cleaned records:         1,682
Records removed:         409
Percentage retained:     80.4%

Changes made:
  ✓ Changed 'Nearby' → 'Yes' in OAK_WILT_CONFIRMED
  ✓ Removed 'Undetermined' and 'Maybe' from OAK_WILT_CONFIRMED
  ✓ Kept only rows with 'Live Oak' in SPECIES

Final OAK_WILT_CONFIRMED values:
OAK_WILT_CONFIRMED
Yes    1672
No       10
Name: count, dtype: int64

Final data quality:
  - All rows have Yes/No for oak wilt confirmation
  - All rows contain Live Oak trees


## 7. Save Cleaned Data

In [8]:
# Save the cleaned dataset
df.to_csv('../data/data_cleaned.csv', index=False)

print("✓ Cleaned data saved to: ../data/data_cleaned.csv")
print(f"✓ Total records in cleaned file: {len(df):,}")

✓ Cleaned data saved to: ../data/data_cleaned.csv
✓ Total records in cleaned file: 1,682


## 8. Verify Cleaned Data

In [9]:
# Load the cleaned file to verify it was saved correctly
df_verify = pd.read_csv('../data/data_cleaned.csv')

print("="*60)
print("VERIFICATION - Loaded cleaned data")
print("="*60)
print(f"Records: {len(df_verify):,}")
print(f"Columns: {len(df_verify.columns)}")
print()
print("OAK_WILT_CONFIRMED values:")
print(df_verify['OAK_WILT_CONFIRMED'].value_counts())
print()
print("Sample of cleaned data:")
df_verify.head(10)

VERIFICATION - Loaded cleaned data
Records: 1,682
Columns: 12

OAK_WILT_CONFIRMED values:
OAK_WILT_CONFIRMED
Yes    1672
No       10
Name: count, dtype: int64

Sample of cleaned data:


,Inspection_ID (unique),MORTALITY_CENTER_ID,INSPECTION_DATE,INSPECTION_DECADE,INSPECTION_YEAR,SPECIES,LATITUDE,LONGITUDE,LANDOWNER_TYPE,OAK_WILT_CONFIRMED,SYMPTOM_VEINAL_NECROSIS,SYMPTOM_MORTALITY_PATTERN
0,NaN,5758.0,8/24/86,1980s,1986,Live Oak,30.318636,-97.766907,Unknown,Yes,NaN,NaN
1,NaN,5723.0,3/11/87,1980s,1987,Live Oak,30.263226,-97.804992,Unknown,Yes,Yes,NaN
2,NaN,5782.0,3/17/88,1980s,1988,Live Oak,30.247574,-97.759786,Unknown,Yes,NaN,NaN
3,NaN,5807.0,5/6/88,1980s,1988,Live Oak,30.229521,-97.859917,Unknown,Yes,Yes,NaN
4,NaN,5689.0,12/6/88,1980s,1988,Live Oak,30.388368,-97.687027,Unknown,Yes,Yes,NaN
5,NaN,5800.0,5/30/89,1980s,1989,Live Oak,30.242403,-97.826803,Unknown,Yes,Yes,Yes
6,NaN,5769.0,7/6/89,1980s,1989,Live Oak,30.356619,-97.767053,Unknown,Yes,NaN,NaN
7,NaN,5776.0,9/21/89,1980s,1989,Live Oak,30.251062,-97.785271,Unknown,Yes,Yes,NaN
8,NaN,5755.0,10/11/89,1980s,1989,Live Oak,30.296323,-97.777386,Unknown,Yes,Yes,NaN
9,NaN,14556.0,10/24/89,1980s,1989,Live Oak,30.199125,-97.818824,Unknown,Yes,Yes,NaN
